In [1]:
from heapq import heappop, heappush

In [2]:
def get_input(filename):
    with open(filename) as f:
        grid = {}
        for i, line in enumerate(f):
            for j, s in enumerate(line.strip()):
                if s in "SE.":
                    grid[i, j] = 1
                if s == "S":
                    source = (i, j)
                elif s == "E":
                    target = (i, j)
    return grid, source, target

def dijkstra(source, target, grid):
    rots = {
        (0, 1): [(-1, 0), (1, 0)],
        (0, -1): [(-1, 0), (1, 0)],
        (1, 0): [(0, -1), (0, 1)],
        (-1, 0): [(0, -1), (0, 1)]
    }
    i, j = source
    di, dj = 0, 1
    cost = {(i, j, di, dj): 0}
    pq = [(0, i, j, di, dj)]
    prev = {}
    while pq:
        c, i, j, di, dj = heappop(pq)
        if (i, j) == target:
            return c, prev, di, dj
        ni, nj = i + di, j + dj
        if (ni, nj) in grid:
            alt = cost[i, j, di, dj] + 1
            if (ni, nj, di, dj) not in cost or alt < cost[(ni, nj, di, dj)]:
                heappush(pq, (alt, ni, nj, di, dj))
                cost[ni, nj, di, dj] = alt
                prev[ni, nj, di, dj] = (i, j, di, dj)
        for ndi, ndj in rots[di, dj]:
            alt = cost[i, j, di, dj] + 1000
            if (i, j, ndi, ndj) not in cost or alt < cost[(i, j, ndi, ndj)]:
                heappush(pq, (alt, i, j, ndi, ndj))
                cost[i, j, ndi, ndj] = alt
                prev[i, j, ndi, ndj] = (i, j, di, dj)


In [3]:
grid, source, target = get_input("16_input.txt")
cost, prev, di, dj = dijkstra(source, target, grid)
print(cost)

91464


In [6]:
def dijkstra2(source, target, grid):
    rots = {
        (0, 1): [(-1, 0), (1, 0)],
        (0, -1): [(-1, 0), (1, 0)],
        (1, 0): [(0, -1), (0, 1)],
        (-1, 0): [(0, -1), (0, 1)]
    }
    i, j = source
    di, dj = 0, 1
    cost = {((i, j, di, dj), tuple()): 0}
    pq = [(0, (i, j, di, dj), tuple())]
    paths = []
    while pq:
        c, u, path = heappop(pq)
        i, j = u[0], u[1]
        di, dj = u[2], u[3]
        if (i, j) == target:
            paths.append(path)
        ni, nj = i + di, j + dj
        if (ni, nj) in grid:
            v = (ni, nj, di, dj)
            alt = cost[(u, path)] + 1
            if (v, path + (v)) not in cost or alt < cost[(v, path + (v))]:
                heappush(pq, (alt, v, path + (v)))
                cost[(v, path + (v))] = alt
        for ndi, ndj in rots[di, dj]:
            w = (i, j, ndi, ndj)
            alt = cost[(u, path)] + 1000
            if (w, path + (w)) not in cost or alt < cost[(w, path + (w))]:
                heappush(pq, (alt, w, path + (w)))
                cost[(w, path + (w))] = alt
    return paths

In [7]:
grid, source, target = get_input("16_testinput.txt")
paths = dijkstra2(source, target, grid)

KeyboardInterrupt: 

In [125]:
targets

[(1, 13, -1, 0)]

In [133]:
stack = [(target, [target])]
paths = []
while stack:
    node, path = stack.pop()
    if node == source:
        paths.append(path[::-1])
        continue
    # if node not in prev:
    #     continue
    for pred in prev[node]:
        stack.append((pred, path + [pred]))

KeyError: (13, 1, 0, 1)

In [130]:
paths

[]

In [119]:
len(set([(i, j) for (i, j, di, dj) in s]))

37

In [134]:
(1, 2) + (3, 4)

(1, 2, 3, 4)